In [1]:
import os
import numpy as np
from PIL import Image
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from collections import defaultdict
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import learning_curve


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "E:\Softwares\ANACONDA\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "E:\Softwares\ANACONDA\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "E:\Softwares\ANACONDA\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "E:\Softwares\ANACONDA\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "E:\Softwares\ANACONDA\lib\site-packages\ipykernel\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "E:\Softwares\ANACONDA\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "E:\Softwares\ANACONDA\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "E:\Softwares\ANACONDA\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "E:\Softwares\ANACONDA\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "E:\Softwares\ANACONDA\lib\site-packages\ipykernel\

AttributeError: _ARRAY_API not found

In [2]:
import matplotlib.pyplot as plt
import xgboost as xgb

#### Read metadata of fish ID, image ID, length, weight, sex, species info

In [5]:
# read meta data
meta_data = pd.read_csv(r".\otoliths\metadata.csv")

In [7]:
meta_data.describe()

,age,length,weight,month,is_male,is_female,is_unknown,is_plaice,is_herring
count,7828.000000,7828.000000,7828.000000,7828.000000,7828.000000,7828.000000,7828.000000,7828.000000,7828.000000
mean,5.842616,0.231746,0.126808,0.684147,0.354497,0.421947,0.223556,0.635795,0.364205
std,2.949646,0.074330,0.107026,0.160089,0.478391,0.493902,0.416655,0.481237,0.481237
min,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,0.180000,0.038000,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,0.250000,0.113030,0.750000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,8.000000,0.287900,0.194692,0.750000,1.000000,1.000000,0.000000,1.000000,1.000000
max,27.000000,0.510000,1.102000,0.750000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Read images

In [8]:
def convert_img_grayscale(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts the images specified in the 'img_path' column to grayscale, resizes them to 64x64.
    Returns a DataFrame with 'uuid' and the flattened pixel values.
    
    Parameters:
        df (pd.DataFrame): DataFrame with columns 'uuid' and 'img_path'.
    
    Returns:
        pd.DataFrame: A new DataFrame with 'uuid' and columns for each pixel in the image.
    """
    # List to store uuid and flattened image data
    uuid_list = []
    pixel_values_list = []
    
    # Loop through each row in the DataFrame
    for _, row in df.iterrows():
        img_path = row['img_path']
        uuid = row['uuid']
        
        try:
            # Open the image using Pillow
            with Image.open(img_path) as img:
                # Resize the image to 64x64
                img_resized = img.resize((64, 64))
                
                # Convert the image to grayscale
                img_gray = img_resized.convert("L")  # 'L' mode stands for grayscale
                
                # img_gray.show()
                # Convert the grayscale image to a numpy array
                img_array = np.array(img_gray)
                
                # Flatten the 2D array to 1D
                img_flattened = img_array.flatten()
                
                # Append the results to the lists
                uuid_list.append(uuid)
                pixel_values_list.append(img_flattened)
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")
            # If error occurs, add None for the image
            uuid_list.append(uuid)
            pixel_values_list.append([None] * 4096)  # Assuming a 64x64 image (4096 pixels)
    
    # Create a new DataFrame with 'uuid' and the flattened image data
    pixel_columns = [f'pixel_{i+1}' for i in range(4096)]  # 64x64 = 4096 pixels
    result_df = pd.DataFrame(pixel_values_list, columns=pixel_columns)
    result_df['uuid'] = uuid_list  # Add 'uuid' column at the end
    
    # Reorder columns to have 'uuid' as the first column
    result_df = result_df[['uuid'] + pixel_columns]
    
    return result_df


def preprocess_img_file(image_folder: str,image_files: list) -> pd.DataFrame:
    # Use a defaultdict to group files by prefix
    prefix_dict = defaultdict(list)
    unique_img_dic = {}

    # create a uuid dictionary with uuid as key andit's otolith images as values
    for image_file in image_files:
        # Extract the UUID from the image filename
        uuid = image_file.split('_')[0]  # Get the part before "_"

        prefix_dict[uuid].append(image_file)

    for uuid, images in prefix_dict.items():
        # Get the first image for each prefix
        if images:
            first_image = images[0]

        # Construct the full image path
            if uuid not in unique_img_dic:
                image_path = os.path.join(image_folder, first_image)
                unique_img_dic[uuid] = image_path

    unique_img_df = pd.DataFrame(list(unique_img_dic.items()), columns=['uuid', 'img_path'])

    gs_img = convert_img_grayscale(unique_img_df) 

    return gs_img

In [9]:
# Path to the image folder
image_folder = r".\otoliths\images"

# List all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')]

In [10]:
# preprocess the images: select the first image of each otolith and convert to grayscale
gs_img = preprocess_img_file(image_folder,image_files)

In [11]:
gs_img.head()

,uuid,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_4087,pixel_4088,pixel_4089,pixel_4090,pixel_4091,pixel_4092,pixel_4093,pixel_4094,pixel_4095,pixel_4096
0,0004210f-e857-4f5a-8cfa-e6cf3b044202,25,25,26,27,27,27,26,26,27,...,26,26,27,26,25,25,26,26,26,27
1,00151248-60eb-4138-93f5-07cda6d070fd,18,5,4,4,5,7,6,6,5,...,6,6,6,6,4,4,4,4,5,5
2,0015eb8b-b631-4440-a1b0-8d83a8d74258,25,25,24,24,24,25,24,25,25,...,24,25,25,25,24,24,24,24,24,24
3,0018a18e-4db9-4d03-9b64-da0c31fb40f2,25,25,24,25,25,25,25,25,25,...,26,25,26,26,25,26,33,26,25,25
4,001c234f-6255-4bbb-9c06-75397d60d141,4,5,4,5,4,5,5,6,6,...,6,5,5,3,4,4,5,5,5,5


In [12]:
gs_img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7828 entries, 0 to 7827
Columns: 4097 entries, uuid to pixel_4096
dtypes: object(1), uint8(4096)
memory usage: 30.6+ MB


#### Create merged_df by merging meta_data and gs_img by 'uuid'

In [14]:
merged_df = pd.merge(meta_data,gs_img, on='uuid',how='left')

In [15]:
merged_df.head(2)

,uuid,fish_id,age,length,weight,month,is_male,is_female,is_unknown,is_plaice,...,pixel_4087,pixel_4088,pixel_4089,pixel_4090,pixel_4091,pixel_4092,pixel_4093,pixel_4094,pixel_4095,pixel_4096
0,388b0180-60bc-440d-bd61-446f8bab884a,CAR-025-62-1466,4,0.22,0.070,0.75,0,1,0,1,...,26,27,26,26,26,26,26,26,26,26
1,81aeec93-b96d-4e14-b3f4-f49c4a8c18c7,CAR-025-72-1692,6,0.20,0.057,0.75,1,0,0,1,...,30,31,29,26,26,26,26,26,27,26


In [16]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7828 entries, 0 to 7827
Columns: 4107 entries, uuid to pixel_4096
dtypes: float64(3), int64(6), object(2), uint8(4096)
memory usage: 31.2+ MB


In [17]:
# Prepare column ID
pixel_columns = [f'pixel_{i+1}' for i in range(4096)] 
cat_vars = ['is_male','is_female', 'is_unknown', 'is_plaice', 'is_herring']
num_vars = ['length','weight','month'] + pixel_columns
metadata_vars = ['is_male','is_female', 'is_unknown', 'is_plaice', 'is_herring','length','weight','month' ]

In [18]:
# cat_vars already onehotencoded, only apply StandardScaler to num_vars
scaler = StandardScaler()
merged_df[num_vars] = scaler.fit_transform(merged_df[num_vars])

In [121]:
merged_df.head(2)

,uuid,fish_id,age,length,weight,month,is_male,is_female,is_unknown,is_plaice,...,pixel_4087,pixel_4088,pixel_4089,pixel_4090,pixel_4091,pixel_4092,pixel_4093,pixel_4094,pixel_4095,pixel_4096
0,388b0180-60bc-440d-bd61-446f8bab884a,CAR-025-62-1466,4,-0.158040,-0.530819,0.411381,0,1,0,1,...,0.637049,0.761876,0.690795,0.694019,0.664656,0.668608,0.691283,0.745199,0.724132,0.797757
1,81aeec93-b96d-4e14-b3f4-f49c4a8c18c7,CAR-025-72-1692,6,-0.427125,-0.652293,0.411381,1,0,0,1,...,0.978145,1.119551,0.963862,0.694019,0.664656,0.668608,0.691283,0.745199,0.815993,0.797757


## Test models on metadata alone (not image data)

In [122]:
# Assign features and labels
X = merged_df[metadata_vars].copy()
y = merged_df['age'].values # Target variable

In [124]:
X.head(2)

,is_male,is_female,is_unknown,is_plaice,is_herring,length,weight,month
0,0,1,0,1,0,-0.158040,-0.530819,0.411381
1,1,0,0,1,0,-0.427125,-0.652293,0.411381


In [123]:
# split dataset, test size 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6262, 8) (1566, 8) (6262,) (1566,)


In [31]:
# split dataset, test size 0.2 (metadata+PCA95% image data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6262, 348) (1566, 348) (6262,) (1566,)


#### Test different regression models, apply RandomizedSearchCV to tune hyperparameters, compare model performance on metadata features

In [125]:
from sklearn.model_selection import RandomizedSearchCV

#### RandomForestRegressor

In [126]:
rfr = RandomForestRegressor(random_state=123)
# parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [10, 20, 50],  
    'max_depth': [5, 10, 20, 50],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],  
    'bootstrap': [True, False],  
}

In [127]:
# RandomizedSearchCV with 3-fold cross-validation and 30 iterations
random_search = RandomizedSearchCV(
    estimator=rfr, 
    param_distributions=param_dist, 
    n_iter=30, 
    cv=3,  # 3-fold cross-validation
    verbose=1, 
    random_state=123
)

In [128]:
# Fit the model to the metadata
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=123),
                   n_iter=30,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, 10, 20, 50],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 20, 50]},
                   random_state=123, verbose=1)

In [129]:
# Get the best hyperparameters
print("Best parameters found: ", random_search.best_params_)
best_rfr = random_search.best_estimator_

Best parameters found:  {'n_estimators': 20, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 10, 'bootstrap': True}


In [139]:
# Evaluate the best model on the test set
y_pred = best_rfr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata alone")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata alone
Mean Squared Error (MSE): 1.40
Root Mean Squared Error: 1.182975057794459
R-squared (R²): 0.85


#### XGBoost

In [131]:
import xgboost as xgb

In [132]:
# Create an XGBoost regressor
xg_reg = xgb.XGBRegressor( random_state=123)
# parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 6, 9, 12],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.1, 0.2],
    'reg_lambda': [0.1, 0.2, 1.0],
}

In [135]:
random_search = RandomizedSearchCV(
    estimator=xg_reg, 
    param_distributions=param_dist, 
    n_iter=20,  
    cv=3,  # 3-fold cross-validation
    verbose=1, 
    random_state=123
)

In [136]:
# Fit the model to the metadata
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=123, ...),
                   n_iter=20,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0, 0.1, 0.2, 0.3],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 6, 9, 12],
                                        'n_estimators': [50, 100, 150, 200],
                                        'reg_alpha': [0, 0.1, 0.2],
                                        'reg_lambda': [0.1, 0.2, 1.0],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=123, verbose=1)

In [137]:
# Get the best hyperparameters
print("Best parameters found: ", random_search.best_params_)
best_xg_reg = random_search.best_estimator_

Best parameters found:  {'subsample': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 0.2, 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.8}


In [140]:
# Evaluate the best model on the test set
y_pred = best_xg_reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata alone")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata alone
Mean Squared Error (MSE): 1.48
Root Mean Squared Error: 1.2183930635840363
R-squared (R²): 0.84


#### ElasticNet

In [141]:
from sklearn.linear_model import ElasticNet

In [142]:
eln = ElasticNet(random_state=123)
# hyperparameter grid search
param_dist = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 10.0],  
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0],  # ratio between L1 and L2 regularization
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=eln,
    param_distributions=param_dist,
    n_iter=20, 
    cv=3,  # 3-fold cross-validation
    random_state=123,
    verbose=1 # Show progress
)

In [143]:
# Fit the model to the data (metadata)
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3, estimator=ElasticNet(random_state=123), n_iter=20,
                   param_distributions={'alpha': [0.1, 0.5, 1.0, 2.0, 10.0],
                                        'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9,
                                                     1.0]},
                   random_state=123, verbose=1)

In [144]:
# Get the best hyperparameters
print("Best parameters found: ", random_search.best_params_)
best_eln = random_search.best_estimator_

Best parameters found:  {'l1_ratio': 0.9, 'alpha': 0.1}


In [145]:
# Evaluate the best model on the test set
y_pred = best_eln.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata alone")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata alone
Mean Squared Error (MSE): 2.10
Root Mean Squared Error: 1.4507276686830155
R-squared (R²): 0.77


#### SVM

In [146]:
from sklearn.svm import LinearSVR

In [147]:
SVR = LinearSVR(random_state=123)

In [148]:
# hyperparameter grid search
param_dist = {
    'epsilon': np.linspace(0.1, 0.5, 5),  
    'max_iter': [10000, 20000, 50000],   
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=SVR,
    param_distributions=param_dist,
    n_iter=15, 
    cv=3,  # 3-fold cross-validation
    random_state=123,
    verbose=1 
)

In [149]:
# Fit the model to the metadata
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


RandomizedSearchCV(cv=3, estimator=LinearSVR(random_state=123), n_iter=15,
                   param_distributions={'epsilon': array([0.1, 0.2, 0.3, 0.4, 0.5]),
                                        'max_iter': [10000, 20000, 50000]},
                   random_state=123, verbose=1)

In [150]:
# Get the best hyperparameters
print("Best parameters found: ", random_search.best_params_)
best_SVR = random_search.best_estimator_

Best parameters found:  {'max_iter': 10000, 'epsilon': np.float64(0.5)}


In [151]:
# Evaluate the best model on the test set
y_pred = best_SVR.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata alone")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata alone
Mean Squared Error (MSE): 1.98
Root Mean Squared Error: 1.408640663757735
R-squared (R²): 0.78


#### Decision Tree

In [152]:
from sklearn.tree import DecisionTreeRegressor

In [153]:
# Hyperparameter grid for Decision Tree
dt_reg = DecisionTreeRegressor(random_state=123)
param_dist = {
    'max_depth': [3, 6, 9, 12],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# RandomizedSearchCV for Decision Tree
random_search = RandomizedSearchCV(
    estimator=dt_reg, 
    param_distributions=param_dist, 
    n_iter=30, 
    cv=3, 
    verbose=1, 
    random_state=123
)

In [154]:
# Fit the model to the metadata
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=123),
                   n_iter=30,
                   param_distributions={'max_depth': [3, 6, 9, 12],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10]},
                   random_state=123, verbose=1)

In [155]:
# Get the best hyperparameters
print("Best parameters found: ", random_search.best_params_)
best_dt_reg = random_search.best_estimator_

Best parameters found:  {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 9}


In [156]:
# Evaluate the best model on the test set
y_pred = best_dt_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata alone")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata alone
Mean Squared Error (MSE): 1.58
Root Mean Squared Error: 1.2568672660799642
R-squared (R²): 0.83


##### Voting ensemble model

In [159]:
from sklearn.ensemble import VotingRegressor

In [160]:
base_models = [
    ('random_forest', best_rfr),
    ('xgboost', best_xg_reg),
    ('SVR', best_SVR),
    ('ElasticNet', best_eln),
    ('decision_tree', best_dt_reg)
]

In [161]:
voting_regressor = VotingRegressor(estimators=base_models)
voting_regressor.fit(X_train, y_train)

VotingRegressor(estimators=[('random_forest',
                             RandomForestRegressor(max_depth=10,
                                                   min_samples_split=5,
                                                   n_estimators=20,
                                                   random_state=123)),
                            ('xgboost',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=0.8, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None,...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=200,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=123, ...)),
                            ('SVR',
                             LinearSVR(epsilon=np.float64(0.5), max_iter=10000,
                                       random_state=123)),
                            ('ElasticNet',
                             ElasticNet(alpha=0.1, l1_ratio=0.9,
                                        random_state=123)),
                            ('decision_tree',
                             DecisionTreeRegressor(max_depth=9,
                                                   random_state=123))])

In [162]:
# Evaluate the voting model on the test set
y_pred = voting_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata alone")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata alone
Mean Squared Error (MSE): 1.50
Root Mean Squared Error: 1.2232412061499216
R-squared (R²): 0.84


#### Ensemble model- Stack using LinearRegression

In [163]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

In [164]:
# Create a Stacking Regressor
stack_model_LR = StackingRegressor(
    estimators=base_models,
    final_estimator=LinearRegression(),
    cv=3, 
    verbose=2
)


In [165]:
# Fit the stacking model
stack_model_LR.fit(X_train, y_train)

StackingRegressor(cv=3,
                  estimators=[('random_forest',
                               RandomForestRegressor(max_depth=10,
                                                     min_samples_split=5,
                                                     n_estimators=20,
                                                     random_state=123)),
                              ('xgboost',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metri...
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=123, ...)),
                              ('SVR',
                               LinearSVR(epsilon=np.float64(0.5),
                                         max_iter=10000, random_state=123)),
                              ('ElasticNet',
                               ElasticNet(alpha=0.1, l1_ratio=0.9,
                                          random_state=123)),
                              ('decision_tree',
                               DecisionTreeRegressor(max_depth=9,
                                                     random_state=123))],
                  final_estimator=LinearRegression(), verbose=2)

In [166]:
# Evaluate the stacking model on the test set
y_pred = stack_model_LR.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata alone")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata alone
Mean Squared Error (MSE): 1.39
Root Mean Squared Error: 1.1797716015334772
R-squared (R²): 0.85


#### Ensemble model- Stack using GradientBoostingRegressor

In [167]:
from sklearn.ensemble import GradientBoostingRegressor

In [168]:
gbr = GradientBoostingRegressor(n_estimators=100, random_state=123)

In [169]:
# Create a Stacking Regressor
stack_model_gbr = StackingRegressor(
    estimators=base_models,
    final_estimator= gbr,
    cv=3, 
    verbose=2
)


In [170]:
# Fit the stackmodel
stack_model_gbr.fit(X_train, y_train)

StackingRegressor(cv=3,
                  estimators=[('random_forest',
                               RandomForestRegressor(max_depth=10,
                                                     min_samples_split=5,
                                                     n_estimators=20,
                                                     random_state=123)),
                              ('xgboost',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metri...
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=123, ...)),
                              ('SVR',
                               LinearSVR(epsilon=np.float64(0.5),
                                         max_iter=10000, random_state=123)),
                              ('ElasticNet',
                               ElasticNet(alpha=0.1, l1_ratio=0.9,
                                          random_state=123)),
                              ('decision_tree',
                               DecisionTreeRegressor(max_depth=9,
                                                     random_state=123))],
                  final_estimator=GradientBoostingRegressor(random_state=123),
                  verbose=2)

In [171]:
# Evaluate the stack model on the test set
y_pred = stack_model_gbr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata alone")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata alone
Mean Squared Error (MSE): 1.39
Root Mean Squared Error: 1.1771126308178697
R-squared (R²): 0.85


## Summary of model performance based on metadata alone

In [172]:
models = {
    'SVR': best_SVR,
    'ElasticNet': best_eln,
    'Decision Tree': best_dt_reg,
    'Random Forest': best_rfr,
    'XGBoost': best_xg_reg,
    'Voting Regressor': voting_regressor,
    'Stacking (Linear Regression)': stack_model_LR, 
    'Stacking (Gradient Boost Regression)': stack_model_gbr
}


In [173]:
mse_values = []
rmse_values = []
r2_values = []

# Loop over models and calculate MSE and RMSE
for name, model in models.items():
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    # Calculate Root Mean Squared Error (RMSE)
    rmse = mse ** 0.5
    # Calculate R²
    r2 = r2_score(y_test, y_pred)    
    # Append the results to the lists
    mse_values.append(mse)
    rmse_values.append(rmse)
    r2_values.append(r2)

In [174]:
results_df = pd.DataFrame({
    'Model': list(models.keys()),
    'Mean Squared Error (MSE)': mse_values,
    'Root Mean Squared Error (RMSE)': rmse_values,
    'R-squared (R²)': r2_values
})

# Display the results
print(results_df)

                                  Model  Mean Squared Error (MSE)  \
0                                   SVR                  1.984269   
1                            ElasticNet                  2.104611   
2                         Decision Tree                  1.579715   
3                         Random Rorest                  1.399430   
4                               XGBoost                  1.484482   
5                      Voting Regressor                  1.496319   
6          Stacking (Linear Regression)                  1.391861   
7  Stacking (Gradient Boost Regression)                  1.385594   

   Root Mean Squared Error (RMSE)  R-squared (R²)  
0                        1.408641        0.782458  
1                        1.450728        0.769264  
2                        1.256867        0.826810  
3                        1.182975        0.846576  
4                        1.218393        0.837251  
5                        1.223241        0.835953  
6             

## Using metadata as input features, the best performace is the stacking model that combines all the base five models () with the meta model Gradient Boost Regression. The MSE is 1.385594, RMSE is 1.177113, and R² is 0.848093.

#### Next, we tested if incorporating fish images can improve model performance

#### We used PCA for dimensionality reduction on image pixel data, didn't use TSNE as it is mainly used for supervised learning

In [21]:
from sklearn.decomposition import PCA

In [22]:
pca = PCA(n_components=0.95)
image_PCA = pca.fit_transform(merged_df[pixel_columns])

print(image_PCA.shape)

(7828, 340)


#### The number of dimensions required to preserve 95% of the image pixel data subset variance is 2.

In [23]:
image_PCA = pd.DataFrame(image_PCA)

In [24]:
image_PCA.columns = [f'Pixel_PCA_{i+1}' for i in range(len(image_PCA.columns))]

In [25]:
image_PCA.head(2)

,Pixel_PCA_1,Pixel_PCA_2,Pixel_PCA_3,Pixel_PCA_4,Pixel_PCA_5,Pixel_PCA_6,Pixel_PCA_7,Pixel_PCA_8,Pixel_PCA_9,Pixel_PCA_10,...,Pixel_PCA_331,Pixel_PCA_332,Pixel_PCA_333,Pixel_PCA_334,Pixel_PCA_335,Pixel_PCA_336,Pixel_PCA_337,Pixel_PCA_338,Pixel_PCA_339,Pixel_PCA_340
0,-36.783400,-4.461431,-0.851298,0.168284,1.173489,0.066053,2.308784,5.282622,-1.018498,-1.974044,...,-0.876237,-0.177330,-0.415348,-0.260030,0.434436,0.170243,0.028277,0.122334,0.317509,-0.068107
1,-24.298016,-14.314338,-6.308319,-0.206874,-7.315224,6.942176,-6.153824,-6.380081,-1.529490,-1.191813,...,0.548163,0.022668,-1.139340,-0.306439,0.567907,0.226244,0.003778,-0.393717,-0.761862,0.065353


#### Create a new dataset "merged_metadata_image_PCA" of metadata and PCA transformed image data image_PCA

In [26]:
merged_metadata_image_PCA = pd.concat([merged_df[['age','is_male','is_female', 'is_unknown', 'is_plaice', 'is_herring', 'length','weight','month']], image_PCA], axis=1)

In [28]:
merged_metadata_image_PCA.shape

(7828, 349)

In [29]:
merged_metadata_image_PCA.describe

<bound method NDFrame.describe of       age  is_male  is_female  is_unknown  is_plaice  is_herring    length  \
0       4        0          1           0          1           0 -0.158040   
1       6        1          0           0          1           0 -0.427125   
2       5        0          1           0          1           0  0.111046   
3       6        1          0           0          1           0 -0.023497   
4       3        0          0           1          1           0 -1.099840   
...   ...      ...        ...         ...        ...         ...       ...   
7823    8        1          0           0          0           1  0.682853   
7824    6        1          0           0          0           1  0.466239   
7825    5        1          0           0          0           1  0.688235   
7826    5        0          1           0          0           1  0.537547   
7827    5        1          0           0          0           1  0.727253   

        weight     month  Pix

In [30]:
# Assign features and labels
X = merged_metadata_image_PCA.drop(columns=['age']).copy()
y = merged_metadata_image_PCA['age'].values # Target variable

In [31]:
# split dataset, test size 0.2 (metadata+PCA95% image data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6262, 348) (1566, 348) (6262,) (1566,)


#### Test different regression models, apply RandomizedSearchCV to tune hyperparms, compare model performance on metadata + PCA95% image data

In [33]:
from sklearn.model_selection import RandomizedSearchCV

#### RandomForestRegressor

In [34]:
rfr = RandomForestRegressor(random_state=123)
# parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [10, 20, 50],  # Number of trees in the forest
    'max_depth': [5, 10, 20, 50],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False],  # Whether bootstrap samples are used when building trees
}

In [35]:
# RandomizedSearchCV with 3-fold cross-validation and 30 iterations
random_search = RandomizedSearchCV(
    estimator=rfr, 
    param_distributions=param_dist, 
    n_iter=30,  # Number of random combinations to try
    cv=3,  # 3-fold cross-validation
    verbose=2,  # Show progress
    random_state=123
)

In [36]:
# Fit the model to the data (metadata and PCA95% image)
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=10; total time=  13.2s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=10; total time=  13.6s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=10; total time=  13.6s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=2, min_samples_split=5, n_estimators=10; total time=  13.1s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=2, min_samples_split=5, n_estimators=10; total time=  13.8s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=2, min_samples_split=5, n_estimators=10; total time=  14.1s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=20; total time=  15.5s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=20; total time=  15.1s
[CV] 

[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time= 1.1min
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time= 1.1min
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=20; total time=   8.0s
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=20; total time=   8.1s
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=20; total time=   8.1s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=2, min_samples_split=5, n_estimators=20; total time=  26.0s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=2, min_samples_split=5, n_estimators=20; total time=  27.1s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=2, min_samples_split=5, n_estimators=20; total time=  27.6s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=4, min_sampl

E:\Softwares\ANACONDA\lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=123),
                   n_iter=30,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, 10, 20, 50],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 20, 50]},
                   random_state=123, verbose=2)

In [37]:
# Get the best hyperparameters
print("Best parameters found: ", random_search.best_params_)
best_rfr = random_search.best_estimator_

Best parameters found:  {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 50, 'bootstrap': True}


In [51]:
# Evaluate the best model on the test set
y_pred = best_rfr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata + image feature")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata + image feature
Mean Squared Error (MSE): 1.56
Root Mean Squared Error: 1.2484240447956287
R-squared (R²): 0.83


#### XGBoost

In [41]:
import xgboost as xgb

In [42]:
# Create an XGBoost regressor
xg_reg = xgb.XGBRegressor( random_state=123)
# parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 6, 9, 12],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.1, 0.2],
    'reg_lambda': [0.1, 0.2, 1.0],
}

In [47]:
random_search = RandomizedSearchCV(
    estimator=xg_reg, 
    param_distributions=param_dist, 
    n_iter=20,  
    cv=3,  # 3-fold cross-validation
    verbose=2, 
    random_state=123
)

In [48]:
# Fit the model to the data (metadata and PCA95% image)
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END colsample_bytree=1.0, gamma=0.3, learning_rate=0.1, max_depth=3, n_estimators=200, reg_alpha=0.1, reg_lambda=0.2, subsample=0.8; total time=   2.1s
[CV] END colsample_bytree=1.0, gamma=0.3, learning_rate=0.1, max_depth=3, n_estimators=200, reg_alpha=0.1, reg_lambda=0.2, subsample=0.8; total time=   2.3s
[CV] END colsample_bytree=1.0, gamma=0.3, learning_rate=0.1, max_depth=3, n_estimators=200, reg_alpha=0.1, reg_lambda=0.2, subsample=0.8; total time=   2.1s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.01, max_depth=6, n_estimators=150, reg_alpha=0.1, reg_lambda=0.1, subsample=1.0; total time=   7.4s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.01, max_depth=6, n_estimators=150, reg_alpha=0.1, reg_lambda=0.1, subsample=1.0; total time=   7.3s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.01, max_depth=6, n_estimators=150, reg_alpha=0.1, reg_lambda=0.1, subsample=1.0; total time=  

[CV] END colsample_bytree=1.0, gamma=0.2, learning_rate=0.01, max_depth=6, n_estimators=200, reg_alpha=0, reg_lambda=1.0, subsample=0.8; total time=  10.0s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_depth=6, n_estimators=200, reg_alpha=0.2, reg_lambda=0.1, subsample=1.0; total time=  10.3s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_depth=6, n_estimators=200, reg_alpha=0.2, reg_lambda=0.1, subsample=1.0; total time=  10.3s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_depth=6, n_estimators=200, reg_alpha=0.2, reg_lambda=0.1, subsample=1.0; total time=  10.3s
[CV] END colsample_bytree=1.0, gamma=0.2, learning_rate=0.1, max_depth=3, n_estimators=200, reg_alpha=0, reg_lambda=1.0, subsample=0.6; total time=   2.4s
[CV] END colsample_bytree=1.0, gamma=0.2, learning_rate=0.1, max_depth=3, n_estimators=200, reg_alpha=0, reg_lambda=1.0, subsample=0.6; total time=   2.3s
[CV] END colsample_bytree=1.0, gamma=0.2, learning_rate=0.1, max_dept

RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=123, ...),
                   n_iter=20,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0, 0.1, 0.2, 0.3],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 6, 9, 12],
                                        'n_estimators': [50, 100, 150, 200],
                                        'reg_alpha': [0, 0.1, 0.2],
                                        'reg_lambda': [0.1, 0.2, 1.0],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=123, verbose=2)

In [49]:
# Get the best hyperparameters
print("Best parameters found: ", random_search.best_params_)
best_xg_reg = random_search.best_estimator_

Best parameters found:  {'subsample': 0.8, 'reg_lambda': 0.2, 'reg_alpha': 0.1, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 1.0}


In [52]:
# Evaluate the best model on the test set
y_pred = best_xg_reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata + image feature")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata + image feature
Mean Squared Error (MSE): 1.59
Root Mean Squared Error: 1.2599469482029326
R-squared (R²): 0.83


#### ElasticNet

In [54]:
from sklearn.linear_model import ElasticNet

In [55]:
eln = ElasticNet(random_state=123)
# hyperparameter grid search
param_dist = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 10.0],  
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0],  # ratio between L1 and L2 regularization
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=eln,
    param_distributions=param_dist,
    n_iter=20, 
    cv=3,  # 3-fold cross-validation
    random_state=123,
    verbose=2 # Show progress
)

In [56]:
# Fit the model to the data (metadata and PCA95% image)
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END ............................alpha=0.5, l1_ratio=0.3; total time=   0.0s
[CV] END ............................alpha=0.5, l1_ratio=0.3; total time=   0.0s
[CV] END ............................alpha=0.5, l1_ratio=0.3; total time=   0.0s
[CV] END ...........................alpha=10.0, l1_ratio=1.0; total time=   0.0s
[CV] END ...........................alpha=10.0, l1_ratio=1.0; total time=   0.0s
[CV] END ...........................alpha=10.0, l1_ratio=1.0; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=1.0; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=1.0; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=1.0; total time=   0.0s
[CV] END ...........................alpha=10.0, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=10.0, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=10.0, 

RandomizedSearchCV(cv=3, estimator=ElasticNet(random_state=123), n_iter=20,
                   param_distributions={'alpha': [0.1, 0.5, 1.0, 2.0, 10.0],
                                        'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9,
                                                     1.0]},
                   random_state=123, verbose=2)

In [57]:
# Get the best hyperparameters
print("Best parameters found: ", random_search.best_params_)
best_eln = random_search.best_estimator_

Best parameters found:  {'l1_ratio': 0.7, 'alpha': 0.1}


In [58]:
# Evaluate the best model on the test set
y_pred = best_eln.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata + image feature")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata + image feature
Mean Squared Error (MSE): 1.86
Root Mean Squared Error: 1.3632190812180824
R-squared (R²): 0.80


#### SVM

In [61]:
from sklearn.svm import LinearSVR

In [62]:
SVR = LinearSVR(random_state=123)

In [70]:
# hyperparameter grid search
param_dist = {
    'epsilon': np.linspace(0.1, 0.5, 5),  # Epsilon parameter (ranges from 0.01 to 0.1)
    'max_iter': [10000, 20000, 50000],      # Maximum iterations
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=SVR,
    param_distributions=param_dist,
    n_iter=15, 
    cv=3,  # 3-fold cross-validation
    random_state=123,
    verbose=1 
)

In [71]:
# Fit the model to the data (metadata and PCA95% image)
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

RandomizedSearchCV(cv=3, estimator=LinearSVR(random_state=123), n_iter=15,
                   param_distributions={'epsilon': array([0.1, 0.2, 0.3, 0.4, 0.5]),
                                        'max_iter': [10000, 20000, 50000]},
                   random_state=123, verbose=1)

In [73]:
# Get the best hyperparameters
print("Best parameters found: ", random_search.best_params_)
best_SVR = random_search.best_estimator_

Best parameters found:  {'max_iter': 50000, 'epsilon': np.float64(0.5)}


In [74]:
# Evaluate the best model on the test set
y_pred = best_SVR.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata + image feature")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata + image feature
Mean Squared Error (MSE): 1.77
Root Mean Squared Error: 1.3321721920984293
R-squared (R²): 0.81


#### Decision Tree

In [75]:
from sklearn.tree import DecisionTreeRegressor

In [79]:
# Hyperparameter grid for Decision Tree
dt_reg = DecisionTreeRegressor(random_state=123)
param_dist = {
    'max_depth': [3, 6, 9, 12],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# RandomizedSearchCV for Decision Tree
random_search = RandomizedSearchCV(
    estimator=dt_reg, 
    param_distributions=param_dist, 
    n_iter=30, 
    cv=3, 
    verbose=1, 
    random_state=123
)

In [80]:
# Fit the model to the data (metadata and PCA95% image)
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=123),
                   n_iter=30,
                   param_distributions={'max_depth': [3, 6, 9, 12],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10]},
                   random_state=123, verbose=1)

In [81]:
# Get the best hyperparameters
print("Best parameters found: ", random_search.best_params_)
best_dt_reg = random_search.best_estimator_

Best parameters found:  {'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 6}


In [82]:
# Evaluate the best model on the test set
y_pred = best_dt_reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata + image feature")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata + image feature
Mean Squared Error (MSE): 1.77
Root Mean Squared Error: 1.3292578022950148
R-squared (R²): 0.81


##### Voting ensemble model

In [103]:
from sklearn.ensemble import VotingRegressor

In [106]:
voting_regressor = VotingRegressor(estimators=base_models)
voting_regressor.fit(X_train, y_train)

E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingRegressor(estimators=[('random_forest',
                             RandomForestRegressor(max_depth=50,
                                                   min_samples_leaf=4,
                                                   min_samples_split=10,
                                                   n_estimators=50,
                                                   random_state=123)),
                            ('xgboost',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=1.0, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=Fal...
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=200,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=123, ...)),
                            ('SVR',
                             LinearSVR(epsilon=np.float64(0.5), max_iter=50000,
                                       random_state=123)),
                            ('ElasticNet',
                             ElasticNet(alpha=0.1, l1_ratio=0.7,
                                        random_state=123)),
                            ('decision_tree',
                             DecisionTreeRegressor(max_depth=6,
                                                   min_samples_leaf=4,
                                                   min_samples_split=5,
                                                   random_state=123))])

In [107]:
# Evaluate the voting model on the test set
y_pred = voting_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata + image feature")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata + image feature
Mean Squared Error (MSE): 1.55
Root Mean Squared Error: 1.2440403312975967
R-squared (R²): 0.83


#### Ensemble model- Stack using LinearRegression

In [87]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

In [100]:
base_models = [
    ('random_forest', best_rfr),
    ('xgboost', best_xg_reg),
    ('SVR', best_SVR),
    ('ElasticNet', best_eln),
    ('decision_tree', best_dt_reg)
]

# Create a Stacking Regressor
stack_model_LR = StackingRegressor(
    estimators=base_models,
    final_estimator=LinearRegression(),
    cv=3, 
    verbose=2
)


In [101]:
# Fit the stacking model
stack_model_LR.fit(X_train, y_train)

E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


StackingRegressor(cv=3,
                  estimators=[('random_forest',
                               RandomForestRegressor(max_depth=50,
                                                     min_samples_leaf=4,
                                                     min_samples_split=10,
                                                     n_estimators=50,
                                                     random_state=123)),
                              ('xgboost',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=1.0, device=None,
                                            early_stopping_rounds=None,
                                            enable_categori...
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=123, ...)),
                              ('SVR',
                               LinearSVR(epsilon=np.float64(0.5),
                                         max_iter=50000, random_state=123)),
                              ('ElasticNet',
                               ElasticNet(alpha=0.1, l1_ratio=0.7,
                                          random_state=123)),
                              ('decision_tree',
                               DecisionTreeRegressor(max_depth=6,
                                                     min_samples_leaf=4,
                                                     min_samples_split=5,
                                                     random_state=123))],
                  final_estimator=LinearRegression(), verbose=2)

In [102]:
# Evaluate the stacking model on the test set
y_pred = stack_model_LR.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata + image feature")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata + image feature
Mean Squared Error (MSE): 1.51
Root Mean Squared Error: 1.2305270831622892
R-squared (R²): 0.83


#### Ensemble model- Stack using GradientBoostingRegressor

In [109]:
from sklearn.ensemble import GradientBoostingRegressor

In [110]:
gbr = GradientBoostingRegressor(n_estimators=100, random_state=123)

In [111]:
# Create a Stacking Regressor
stack_model_gbr = StackingRegressor(
    estimators=base_models,
    final_estimator= gbr,
    cv=3, 
    verbose=2
)


In [112]:
# Fit the stackmodel
stack_model_gbr.fit(X_train, y_train)

E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
E:\Softwares\ANACONDA\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


StackingRegressor(cv=3,
                  estimators=[('random_forest',
                               RandomForestRegressor(max_depth=50,
                                                     min_samples_leaf=4,
                                                     min_samples_split=10,
                                                     n_estimators=50,
                                                     random_state=123)),
                              ('xgboost',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=1.0, device=None,
                                            early_stopping_rounds=None,
                                            enable_categori...
                                            num_parallel_tree=None,
                                            random_state=123, ...)),
                              ('SVR',
                               LinearSVR(epsilon=np.float64(0.5),
                                         max_iter=50000, random_state=123)),
                              ('ElasticNet',
                               ElasticNet(alpha=0.1, l1_ratio=0.7,
                                          random_state=123)),
                              ('decision_tree',
                               DecisionTreeRegressor(max_depth=6,
                                                     min_samples_leaf=4,
                                                     min_samples_split=5,
                                                     random_state=123))],
                  final_estimator=GradientBoostingRegressor(random_state=123),
                  verbose=2)

In [113]:
# Evaluate the stack model on the test set
y_pred = stack_model_gbr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print("Performance evaluation: metadata + image feature")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f'Root Mean Squared Error: {rmse}')
print(f"R-squared (R²): {r2:.2f}")

Performance evaluation: metadata + image feature
Mean Squared Error (MSE): 1.61
Root Mean Squared Error: 1.2707093018445996
R-squared (R²): 0.82


## Summary of model performance based on metadata+95%PCA pixel data

In [117]:
models = {
    'SVR': best_SVR,
    'ElasticNet': best_eln,
    'Decision Tree': best_dt_reg,
    'Random Forest': best_rfr,
    'XGBoost': best_xg_reg,
    'Voting Regressor': voting_regressor,
    'Stacking (Linear Regression)': stack_model_LR, 
    'Stacking (Gradient Boost Regression)': stack_model_gbr
}


In [118]:
mse_values = []
rmse_values = []
r2_values = []

# Loop over models and calculate MSE and RMSE
for name, model in models.items():
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    # Calculate Root Mean Squared Error (RMSE)
    rmse = mse ** 0.5
    # Calculate R²
    r2 = r2_score(y_test, y_pred)    
    # Append the results to the lists
    mse_values.append(mse)
    rmse_values.append(rmse)
    r2_values.append(r2)

In [119]:
results_df = pd.DataFrame({
    'Model': list(models.keys()),
    'Mean Squared Error (MSE)': mse_values,
    'Root Mean Squared Error (RMSE)': rmse_values,
    'R-squared (R²)': r2_values
})

# Display the results
print(results_df)

                                  Model  Mean Squared Error (MSE)  \
0                                   SVR                  1.774683   
1                            ElasticNet                  1.858366   
2                         Decision Tree                  1.766926   
3                         Random Rorest                  1.558563   
4                               XGBoost                  1.587466   
5                      Voting Regressor                  1.547636   
6          Stacking (Linear Regression)                  1.514197   
7  Stacking (Gradient Boost Regression)                  1.614702   

   Root Mean Squared Error (RMSE)  R-squared (R²)  
0                        1.332172        0.805435  
1                        1.363219        0.796261  
2                        1.329258        0.806286  
3                        1.248424        0.829129  
4                        1.259947        0.825961  
5                        1.244040        0.830327  
6             

## Using metadata + 95%PCA reduced image data as input features, the best performace is the stacking model that combines five base models (SVR, ElasticNet, Random Forest, and XGBoost) with the meta model Linear Regression. The MSE is 1.514197, RMSE is 1.230527, and R² is 0.833993.
## The best model based on metadata alone appears to perform better than the best model based on metadata + PCA transformed image data. Next, we tested more advanced models to leverage the use of image data.